# 如何使用父文档检索器

在拆分文档以进行[检索](/docs/concepts/retrieval/)时，通常存在相互冲突的需求：

1. 您可能希望文档较小，以便其嵌入能够最准确地反映其含义。如果文档太长，嵌入可能会丢失含义。
2. 您希望文档足够长，以便保留每个块的上下文。

`ParentDocumentRetriever` 通过拆分和存储小型数据块来达到这种平衡。在检索过程中，它首先获取小型数据块，然后查找这些数据块的父 ID，并返回那些较大的文档。

请注意，“父文档”是指小型数据块源自的文档。这可以是整个原始文档或一个较大的数据块。

In [1]:
from langchain.retrievers import ParentDocumentRetriever

In [2]:
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
loaders = [
    TextLoader("paul_graham_essay.txt"),
    TextLoader("state_of_the_union.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

## 检索完整文档

在此模式下，我们要检索完整文档。因此，我们只需指定一个子 [splitter](/docs/concepts/text_splitters/)。

In [4]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [5]:
retriever.add_documents(docs, ids=None)

这就应该产生两个键，因为我们添加了两个文档。

In [6]:
list(store.yield_keys())

['9a63376c-58cc-42c9-b0f7-61f0e1a3a688',
 '40091598-e918-4a18-9be0-f46413a95ae4']

现在让我们调用向量存储搜索功能——我们应该会看到它返回了小的文本块（因为我们存储的就是小的文本块）。

In [7]:
sub_docs = vectorstore.similarity_search("justice breyer")

In [8]:
print(sub_docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


现在，让我们从整体检索器中获取信息。这应该会返回大型文档，因为它会返回包含较小块的文档。

In [9]:
retrieved_docs = retriever.invoke("justice breyer")

In [10]:
len(retrieved_docs[0].page_content)

38540

## 检索更大的块

有时，完整的文档可能太大，我们不希望按原样检索它们。在这种情况下，我们真正想做的是先将原始文档分割成较大的块，然后再将其分割成较小的块。我们索引较小的块，但在检索时，我们检索较大的块（但仍然不是完整的文档）。

In [11]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()

In [12]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [13]:
retriever.add_documents(docs)

现在我们可以看到有远远不止两份文档了——这些是更大的块。

In [14]:
len(list(store.yield_keys()))

66

让我们确保底层向量存储仍然检索小型分块。

In [15]:
sub_docs = vectorstore.similarity_search("justice breyer")

In [16]:
print(sub_docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


In [18]:
retrieved_docs = retriever.invoke("justice breyer")

In [19]:
len(retrieved_docs[0].page_content)

1849

In [20]:
print(retrieved_docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal publi